In [8]:
import polars as pl

In [9]:
group_by_1e7_csv = "../data/h2o/groupby-datagen_1e7_1e2_0_0/csv/G1_1e7_1e2_0_0.csv"
group_by_1e7_parquet = "../data/h2o/polars/G1_1e7_1e2_0_0.parquet"

In [10]:
with pl.StringCache():
    df_csv = pl.read_csv(
        group_by_1e7_csv,
        dtype={
            "id1": pl.Utf8,
            "id2": pl.Utf8,
            "id3": pl.Utf8,
            "id4": pl.Int32,
            "id5": pl.Int32,
            "id6": pl.Int32,
            "v1": pl.Int32,
            "v2": pl.Int32,
            "v3": pl.Float64,
        },
        low_memory=True,
    ).with_columns(
        [
            pl.col("id1").cast(pl.Categorical),
            pl.col("id2").cast(pl.Categorical),
            pl.col("id3").cast(pl.Categorical),
        ]
    )

In [11]:
df_csv_lazy = df_csv.lazy()

In [12]:
df_scan_parquet_lazy = pl.scan_parquet(group_by_1e7_parquet, cache=False, parallel=True)

## q1: sum v1 by id1

In [13]:
%%time
df_csv_lazy.groupby("id1").agg(pl.sum("v1")).collect()

CPU times: user 136 ms, sys: 80 ms, total: 216 ms
Wall time: 52 ms


id1,v1_sum
categorical,i32
"""id033""",298180
"""id064""",297115
"""id017""",301744
"""id004""",299708
"""id015""",301738
"""id094""",300798
"""id097""",299226
"""id073""",298940
"""id067""",299895


In [14]:
%%time
df_scan_parquet_lazy.groupby("id1").agg(pl.sum("v1")).collect()

CPU times: user 442 ms, sys: 83.3 ms, total: 525 ms
Wall time: 286 ms


id1,v1_sum
categorical,i32
"""id065""",300249
"""id100""",298958
"""id042""",300963
"""id048""",299352
"""id099""",301060
"""id084""",301421
"""id022""",298647
"""id044""",300415
"""id043""",299557


## q2: sum v1 by id1:id2

In [15]:
%%time
df_csv_lazy.groupby(["id1", "id2"]).agg(pl.sum("v1")).collect()

CPU times: user 607 ms, sys: 29.1 ms, total: 636 ms
Wall time: 116 ms


id1,id2,v1_sum
categorical,categorical,i32
"""id055""","""id055""",3083
"""id042""","""id010""",2942
"""id042""","""id005""",2882
"""id016""","""id030""",3003
"""id042""","""id082""",2816
"""id016""","""id070""",3079
"""id062""","""id100""",3178
"""id065""","""id040""",3207
"""id062""","""id094""",2844


In [16]:
%%time
df_scan_parquet_lazy.groupby(["id1", "id2"]).agg(pl.sum("v1")).collect()

CPU times: user 1.14 s, sys: 48.3 ms, total: 1.19 s
Wall time: 373 ms


id1,id2,v1_sum
categorical,categorical,i32
"""id042""","""id076""",3014
"""id064""","""id051""",3000
"""id098""","""id034""",3119
"""id065""","""id082""",2901
"""id044""","""id060""",3074
"""id062""","""id073""",2915
"""id055""","""id071""",2716
"""id095""","""id058""",2972
"""id042""","""id015""",2947


## q3: sum v1 mean v3 by id3

In [17]:
%%time
df_csv_lazy.groupby("id3").agg([pl.sum("v1"), pl.mean("v3")]).collect()

CPU times: user 662 ms, sys: 83.2 ms, total: 745 ms
Wall time: 118 ms


id3,v1_sum,v3_mean
categorical,i32,f64
"""id0000012562""",298,48.165062
"""id0000029266""",286,48.895128
"""id0000017767""",298,47.164840
"""id0000025056""",260,47.923873
"""id0000029379""",303,48.470385
"""id0000036416""",308,51.217770
"""id0000036006""",320,51.988006
"""id0000004197""",252,44.302972
"""id0000038457""",288,50.528941


In [18]:
%%time
df_scan_parquet_lazy.groupby("id3").agg([pl.sum("v1"), pl.mean("v3")]).collect()

CPU times: user 1.28 s, sys: 217 ms, total: 1.5 s
Wall time: 698 ms


id3,v1_sum,v3_mean
categorical,i32,f64
"""id0000089290""",350,46.744325
"""id0000029076""",271,49.050465
"""id0000049271""",278,47.571874
"""id0000041923""",263,49.480641
"""id0000087262""",241,53.424145
"""id0000062191""",257,52.075748
"""id0000080301""",278,49.669852
"""id0000054355""",281,48.681688
"""id0000079397""",288,49.483685


## q4: mean v1:v3 by id4

In [19]:
%%time
df_csv_lazy.groupby("id4").agg([pl.mean("v1"), pl.mean("v2"), pl.mean("v3")]).collect()

CPU times: user 236 ms, sys: 56.4 ms, total: 292 ms
Wall time: 47.5 ms


id4,v1_mean,v2_mean,v3_mean
i32,f64,f64,f64
77,3.002807,8.004582,50.118955
48,2.993247,8.013077,50.069034
40,2.992388,8.011892,50.034023
96,2.996474,7.983614,49.889232
47,3.000261,8.013840,49.974275
42,3.007564,7.977498,49.973327
25,3.000725,7.986656,49.985759
32,2.999519,7.992296,50.017648
10,3.006348,8.018560,50.092483


In [20]:
%%time
df_scan_parquet_lazy.groupby("id4").agg([pl.mean("v1"), pl.mean("v2"), pl.mean("v3")]).collect()

CPU times: user 469 ms, sys: 232 ms, total: 702 ms
Wall time: 198 ms


id4,v1_mean,v2_mean,v3_mean
i32,f64,f64,f64
79,3.006719,7.990951,50.109378
30,2.998215,8.010432,49.925469
91,2.996868,8.010745,50.120128
78,2.998551,8.006425,49.935622
72,2.994291,7.969376,50.088017
5,2.999491,8.013990,50.005535
48,2.993247,8.013077,50.069034
55,3.000340,7.995121,49.921878
42,3.007564,7.977498,49.973327


## q5: sum v1:v3 by id6

In [21]:
%%time
df_csv_lazy.groupby("id6").agg([pl.sum("v1"), pl.sum("v2"), pl.sum("v3")]).collect()

CPU times: user 974 ms, sys: 20.6 ms, total: 994 ms
Wall time: 146 ms


id6,v1_sum,v2_sum,v3_sum
i32,i32,i32,f64
3128,276,715,4360.263866
68912,289,826,4975.177407
46664,322,788,4767.362350
84584,364,979,5495.240547
18800,348,938,6344.946306
73816,342,917,5291.220126
42472,244,549,3672.188740
53240,339,903,5324.468629
30992,322,862,5693.327629


In [22]:
%%time
df_scan_parquet_lazy.groupby("id6").agg([pl.sum("v1"), pl.sum("v2"), pl.sum("v3")]).collect()

CPU times: user 1.15 s, sys: 190 ms, total: 1.34 s
Wall time: 284 ms


id6,v1_sum,v2_sum,v3_sum
i32,i32,i32,f64
60200,309,883,5170.283566
30600,324,839,5414.163281
79272,281,702,4888.298325
73504,347,1002,5795.595860
94272,308,918,5725.067084
5040,259,627,4295.501026
84720,260,747,4928.266592
84344,336,890,5865.200022
66472,288,637,4581.096463


## q6: median v3 sd v3 by id4 id5

In [23]:
%%time
df_csv_lazy.groupby(["id4", "id5"]).agg(
    [pl.median("v3").alias("v3_median"), pl.std("v3").alias("v3_std")]
).collect()

CPU times: user 1.18 s, sys: 27.5 ms, total: 1.21 s
Wall time: 187 ms


id4,id5,v3_median,v3_std
i32,i32,f64,f64
16,46,50.167760,28.617277
24,24,49.565027,29.473630
72,37,50.382813,28.488445
48,19,50.564045,29.360585
16,18,46.840431,28.473162
72,44,52.276038,29.034720
32,21,48.584525,27.578023
8,29,50.777353,29.599871
56,75,50.538039,29.038156


In [24]:
%%time
df_scan_parquet_lazy.groupby(["id4", "id5"]).agg(
    [pl.median("v3").alias("v3_median"), pl.std("v3").alias("v3_std")]
).collect()

CPU times: user 1.33 s, sys: 145 ms, total: 1.48 s
Wall time: 312 ms


id4,id5,v3_median,v3_std
i32,i32,f64,f64
88,64,49.653749,29.731343
80,10,49.164125,28.962593
80,37,49.369015,29.047004
8,96,48.484435,27.941251
8,95,50.254366,28.556056
80,91,50.598449,28.820733
88,95,47.770133,28.905362
80,93,50.830674,28.634471
32,39,49.323161,28.154582


## q7: max v1 - min v2 by id3

In [25]:
%%time
df_csv_lazy.groupby("id3").agg([(pl.max("v1") - pl.min("v2")).alias("range_v1_v2")]).collect()

CPU times: user 726 ms, sys: 15.2 ms, total: 741 ms
Wall time: 117 ms


id3,range_v1_v2
categorical,i32
"""id0000030229""",4
"""id0000014305""",4
"""id0000086928""",4
"""id0000093927""",4
"""id0000084325""",4
"""id0000065696""",4
"""id0000025783""",4
"""id0000088487""",4
"""id0000049960""",4


In [26]:
%%time
df_scan_parquet_lazy.groupby("id3").agg([(pl.max("v1") - pl.min("v2")).alias("range_v1_v2")]).collect()

CPU times: user 1.21 s, sys: 50.1 ms, total: 1.27 s
Wall time: 529 ms


id3,range_v1_v2
categorical,i32
"""id0000060203""",4
"""id0000056933""",4
"""id0000090182""",4
"""id0000089839""",4
"""id0000010244""",4
"""id0000042954""",4
"""id0000050808""",4
"""id0000067147""",4
"""id0000074786""",4


## q8: largest two v3 by id6

In [27]:
%%time
df_csv_lazy.drop_nulls("v3").sort("v3", reverse=True).groupby("id6").agg(
    pl.col("v3").head(2).alias("largest2_v3")
).explode("largest2_v3").collect()

CPU times: user 2.98 s, sys: 278 ms, total: 3.26 s
Wall time: 678 ms


id6,largest2_v3
i32,f64
63056,99.811883
63056,99.71133
54136,99.623424
54136,97.207863
14720,99.307969
14720,96.527665
36064,99.26675
36064,97.019835
21792,98.938464


In [28]:
%%time
df_scan_parquet_lazy.drop_nulls("v3").sort("v3", reverse=True).groupby("id6").agg(
    pl.col("v3").head(2).alias("largest2_v3")
).explode("largest2_v3").collect()

CPU times: user 5.95 s, sys: 765 ms, total: 6.71 s
Wall time: 1.56 s


id6,largest2_v3
i32,f64
25168,98.849642
25168,97.507998
92696,99.788259
92696,99.35893
64864,97.698821
64864,97.646614
77064,99.306127
77064,98.696731
46096,99.476225


## q9: regression v1 v2 by id2 id4

In [29]:
%%time
df_csv_lazy.groupby(["id2","id4"]).agg((pl.pearson_corr("v1","v2")**2).alias("r2")).collect()

CPU times: user 543 ms, sys: 16.7 ms, total: 560 ms
Wall time: 209 ms


id2,id4,r2
categorical,i32,f64
"""id065""",30,0.000441
"""id095""",34,0.000036
"""id064""",86,0.000036
"""id055""",64,0.000459
"""id055""",55,0.000141
"""id038""",48,0.001819
"""id062""",38,0.000129
"""id042""",83,0.001547
"""id065""",69,0.001782


In [30]:
%%time
df_scan_parquet_lazy.groupby(["id2","id4"]).agg((pl.pearson_corr("v1","v2")**2).alias("r2")).collect()

CPU times: user 974 ms, sys: 48.7 ms, total: 1.02 s
Wall time: 469 ms


id2,id4,r2
categorical,i32,f64
"""id085""",49,0.000023
"""id043""",92,0.001683
"""id091""",34,0.000661
"""id016""",80,0.000149
"""id091""",3,0.000269
"""id037""",2,0.002763
"""id083""",62,0.000869
"""id008""",24,0.000542
"""id082""",64,0.006042


## q10: sum v3 count by id1:id6

In [31]:
%%time
df_csv_lazy.groupby(["id1","id2","id3","id4","id5","id6"]).agg([pl.sum("v3").alias("v3"), pl.count("v1").alias("count")]).collect()

CPU times: user 4.67 s, sys: 1.43 s, total: 6.1 s
Wall time: 1.33 s


id1,id2,id3,id4,id5,id6,v3,count
categorical,categorical,categorical,i32,i32,i32,f64,u32
"""id085""","""id083""","""id0000044436""",45,93,38666,62.471425,1
"""id086""","""id031""","""id0000014215""",81,75,32894,94.037358,1
"""id014""","""id034""","""id0000013407""",39,7,50263,39.952116,1
"""id025""","""id080""","""id0000035278""",30,59,8653,55.566843,1
"""id098""","""id099""","""id0000048691""",60,99,10263,57.484029,1
"""id086""","""id032""","""id0000039701""",44,54,83682,31.607898,1
"""id044""","""id099""","""id0000026984""",33,54,91869,61.892416,1
"""id016""","""id090""","""id0000052050""",43,90,21587,41.579015,1
"""id053""","""id033""","""id0000005774""",27,76,73684,72.821838,1


In [32]:
%%time
df_scan_parquet_lazy.groupby(["id1","id2","id3","id4","id5","id6"]).agg([pl.sum("v3").alias("v3"), pl.count("v1").alias("count")]).collect()

CPU times: user 6.17 s, sys: 1.93 s, total: 8.1 s
Wall time: 2.03 s


id1,id2,id3,id4,id5,id6,v3,count
categorical,categorical,categorical,i32,i32,i32,f64,u32
"""id018""","""id063""","""id0000087166""",91,12,67493,34.468358,1
"""id077""","""id079""","""id0000027585""",19,10,20781,85.500698,1
"""id041""","""id033""","""id0000059038""",26,90,24582,53.286813,1
"""id082""","""id046""","""id0000076575""",94,31,25325,21.75352,1
"""id023""","""id092""","""id0000022676""",37,92,42309,62.379427,1
"""id032""","""id096""","""id0000085849""",74,48,22273,28.15394,1
"""id071""","""id001""","""id0000026391""",34,59,36328,38.58325,1
"""id011""","""id065""","""id0000049974""",44,61,63369,5.233783,1
"""id084""","""id029""","""id0000092595""",71,2,55323,35.968176,1
